In [9]:
from sklearn import svm, metrics
import glob, os.path, re, json


# 텍스트를 읽어 들이고 출현 빈도 조사하기
def check_freq(fname):
    name = os.path.basename(fname)
    lang = re.match(r'^[a-z]{2,}', name).group()
    with open(fname, "r", encoding="utf-8") as f:
        text = f.read()
    
    # 모두 소문자를 기준으로 빈도수 계산
    text = text.lower()

    # 알파벳 26자를 인덱싱해서 cnt 변수 0으로 초기화
    cnt = [0 for n in range(0, 26)]
    
    # 시작값 a의 아스키코드(97)와 끝값 z의 아스키코드 (122) 구하기
    code_a = ord("a")
    code_z = ord("z")
    
    # 알파벳 출현 횟수 구하기
    # 텍스트에서 문자 단위로 읽어 아스키값 범위 안에 있을 때 해당 인덱스의 카운터 증가
    for ch in text:
        n = ord(ch)
        if code_a <= n <= code_z: # a~z 사이에 있을 때
            cnt[n - code_a] += 1
            
    # 정규화를 위해 전체 빈도수 합산
    total = sum(cnt)
    
    # 정규화(normalization)하기 - 특징량의 빈도수가 다르므로 1을 기준으로 구하기 위함
    freq = list(map(lambda n: n / total, cnt))
    
    return (freq, lang)
    
# 각 파일 처리하기
def load_files(path):
    freqs = []
    labels = []
    file_list = glob.glob(path)
    
    for fname in file_list:
        r = check_freq(fname)
        
        # freq_tuple.txt 파일에서 r 변수 내용 확인
        
        freqs.append(r[0])
        labels.append(r[1])
        
    return {"freqs":freqs, "labels":labels}

# 학습용 데이터
data = load_files("./lang/train/*.txt")

# 문제 테스트 데이터
test = load_files("./lang/test/*.txt")

# 이후를 대비해서 JSON으로 결과 저장하기
with open("./lang/freq.json", "w", encoding="utf-8") as fp:
    json.dump([data, test], fp)

# 학습하기 --- (※4)
clf = svm.SVC()
clf.fit(data["freqs"], data["labels"])

# 예측하기 --- (※5)
predict = clf.predict(test["freqs"])

# 결과 테스트하기 --- (※6)
ac_score = metrics.accuracy_score(test["labels"], predict)
cl_report = metrics.classification_report(test["labels"], predict)

print("정답률 =", ac_score)
print("리포트 =")
print(cl_report)

정답률 = 0.875
리포트 =
             precision    recall  f1-score   support

         en       0.67      1.00      0.80         2
         fr       1.00      1.00      1.00         2
         id       1.00      0.50      0.67         2
         tl       1.00      1.00      1.00         2

avg / total       0.92      0.88      0.87         8

